Run this cell, wait for the kernel to restart and then run the next cell:

In [ ]:
!pip install -q git+https://github.com/conda-incubator/condacolab.git@0.1.x
import condacolab
condacolab.install()

In [ ]:
!mamba install -q pip scipy networkx conda-forge::boost aivant::openstructure anaconda::py-boost

In [ ]:
!ost --version

In [ ]:
import ost
print(ost.__version__)

In [ ]:
# Check conop
from ost import conop
print(conop.GetDefaultLib().FindCompound("A1LU6").name)

In [ ]:
from ost import io
# Target
target_structure = io.LoadPDB("9CE4/9CE4_A.pdb")
target_ligand = io.LoadSDF("9CE4/9CE4_lig.sdf")
# Model
model_structure = io.LoadPDB("9CE4/996_model1.pdb")
model_ligand = io.LoadSDF("9CE4/996_model1_ligand1.sdf")


In [ ]:
# Cleanup a copy of the protein structures
from ost import conop
from ost.mol.alg import Molck, MolckSettings
cleaned_model_structure = model_structure.Copy()
cleaned_target_structure = target_structure.Copy()
molck_settings = MolckSettings(rm_unk_atoms=True,  # Remove unknown atoms
                               rm_non_std=False,  # Keep non standard residues
                               rm_hyd_atoms=True,  # Remove Hydrogens
                               rm_oxt_atoms=False,  # Keep terminal oxygens
                               rm_zero_occ_atoms=False,  # Keep atoms with 0 occupancy
                               colored=False,
                               map_nonstd_res=False,
                               assign_elem=True)
Molck(cleaned_model_structure, conop.GetDefaultLib(), molck_settings)
Molck(cleaned_target_structure, conop.GetDefaultLib(), molck_settings)

In [ ]:
# Cleanup the ligands
# Remove hydrogens
cleaned_model_ligand = model_ligand.Select("ele != H and ele != D")
cleaned_target_ligand = target_ligand.Select("ele != H and ele != D")


In [ ]:
from ost.mol.alg.ligand_scoring import LDDTPLIScorer, SCRMSDScorer

# Score with LDDT-PLI
scorer = LDDTPLIScorer(
    target = cleaned_target_structure,
    target_ligands = [cleaned_target_ligand],
    model = cleaned_model_structure,
    model_ligands = [cleaned_model_ligand],
    # Extra arguments
    resnum_alignments=True,
    )
chain_name = cleaned_model_ligand.chains[0].name
residue_number = cleaned_model_ligand.residues[0].number
print("LDDT-PLI: ", scorer.score[chain_name][residue_number])

# Score with RMSD
scorer = SCRMSDScorer(
    target = cleaned_target_structure,
    target_ligands = [cleaned_target_ligand],
    model = cleaned_model_structure,
    model_ligands = [cleaned_model_ligand],
    # Extra arguments
    resnum_alignments=True,
    )
print("BiSyRMSD: ", scorer.score[chain_name][residue_number])
print("LDDT-LP: ", scorer.aux[chain_name][residue_number]["lddt_lp"])


In [ ]:
! ost compare-ligand-structures \
    --reference 9CE4/9CE4_A.pdb \
    --reference-ligands 9CE4/9CE4_lig.sdf \
    --model 9CE4/996_model1.pdb \
    --model-ligands 9CE4/996_model1_ligand1.sdf \
    --output T1186LG350_1.json \
    --lddt-pli \
    --rmsd